# Axis Styles

This notebook aims at investigating the distances between different axis designs to see what the size of the subspace is they span.
I would also like to see, how including the original prompt into these emebddings changes the distances!

In [4]:
import torch
import sys
sys.path.append("../")
from prototype.user_profile_host import UserProfileHost

In [5]:
# A function to measure the average distance between tensors

def average_cosine_distance(tensor):
    # Assume last token is EOS embedding (shape [N, D])
    eos_embeds = tensor[:, -1, :]
    sims = torch.nn.functional.cosine_similarity(eos_embeds.unsqueeze(1), eos_embeds.unsqueeze(0), dim=-1)
    n = eos_embeds.shape[0]
    return (1 - sims).sum() / (n * (n - 1))

In [6]:
original_prompt = 'a cat'

res = []

for axis_style in ['random', 'simple', 'ordered', 'complex']:
    for use in [False, True]:
        # Create a UserProfileHost with the respective prompt
        user_profile_host = UserProfileHost(
            original_prompt=original_prompt,
            cache_dir='../cache/',
            axis_style=axis_style,
            use_embedding_center=use,
            n_embedding_axis=20
        )

        # Extract Embeddings of axis
        embeddings = user_profile_host.embedding_axis

        # Average Variance of embeddings
        avg_dist = average_cosine_distance(embeddings)

        # Measure Variance of these embeddings
        res.append([axis_style, use, round(avg_dist.item(), 4)])
print('Style', 'With OG Prompt', 'Diversity')
res

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 12.47it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (109 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', unreal engine 5 , ultra sharp focus , intricate artwork masterpiece , ominous , golden ratio , highly detailed , vibrant , production cinematic character render , ultra high quality model']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['_ stil , wide shot']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['k']
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 12.31it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (106 > 77). Running this sequence

Style With OG Prompt Diversity


[['random', False, 0.6816],
 ['random', True, 0.789],
 ['simple', False, 0.5231],
 ['simple', True, 0.5966],
 ['ordered', False, 0.5638],
 ['ordered', True, 0.5786],
 ['complex', False, 0.5301],
 ['complex', True, 0.5301]]